In [1]:
import spacy
import gensim
import os
import pandas as pd
from gensim.models import KeyedVectors
from joblib import Parallel, delayed
import itertools
import numpy as np
import pickle

data

In [ ]:
%%time
dataset = pd.read_parquet('data/word_embeddings.parquet.gz')
dataset.head()

## tf-idf

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [31]:
corpus = dataset['text'].to_list()
corpus[:1]

['先月24日深夜、プロゴルファー・石川遼が、マネジメント会社「ケーアイ企画」を通し、一般女性との交際を発表した。異例の発表となった石川の交際。マスコミ各社に送られたFAXには、「限度を超えた取材活動が行われた」といった但し書きも含まれ、取材に対する抗議やけん制の目的もあったのだろう、同27日には、石川の交際を報じた「女性セブン」が発売された一件で、石川の父・勝美氏と同誌の間にひと悶着があったとも伝えられている。そんな折、1日発売「週刊アサヒ芸能」（11/10号）の「NEWS SHOT!」のコーナーでは、石川遼交際発表に至る、水面下で行われた石川サイドとマスコミの攻防を伝えている。石川の地元・埼玉県松伏町で取材活動をしていたという「女性セブン」のカメラマンに対し、怒りの電話をかけてまくしたてたという勝美氏。同誌にコメントしたスポーツ記者は、「勝美さんは意に沿わぬマスコミに圧力をかけることで有名ですが、さすがにセブンの記事を止めることはできなかった。それで、記事が出る前に公表してしまおうということになったようです」と明かす。また、「夏頃には『石川が女性を妊娠させた』『隠し子がいる』という怪文書がマスコミやゴルフ関係者の間に出回った」（同誌より）という石川の周辺事情について、同じく同誌にコメントするゴルフ担当記者は「相手の女性についてはモデル、プロの女性と2説ありました。『産んでもらって自分が育てる』と主張する石川に、勝美さんや広告代理店が頭を抱えているという内容。各社ともウラ取りに走ったのですが、今のところ、どこも確証を得られていないようです」と説明する。今オフには結婚説もあるという石川。前出のゴルフ関係者は「勝美さんは、『時期尚早』と考えているが、公表前には関係各所に挨拶回りもしていたようですから、反対というわけでもない。結婚を機に子離れしてくれると、我々マスコミも助かるんですどね（笑）」と語っている。・アサヒ芸能 ［モバイル版］ - モバイルからのアクセスのみ・週刊アサヒ芸能［BookLive!］']

In [53]:
%%time
# コーパス作成
summary = dataset.groupby('media', as_index=False)['text'].sum()
summary.head()

CPU times: user 51.3 s, sys: 51.8 ms, total: 51.4 s
Wall time: 51.4 s


,media,text
0,dokujo-tsushin,先日、同窓会の幹事グループに加わったサキさん（独身／41歳・会社員）。仲が良かった男友達・K...
1,it-life-hack,Facebookでは、写真に友達をタグ付けすることで、そこに写っているのが誰なのかを表すこと...
2,kaden-channel,一見なにかわからないこの固まり、実はUSBだ。書いている数字はこのUSBの容量と重量を示して...
3,livedoor-homme,「女性はデジタルガジェットに弱い」というイメージは、スマートフォンの普及もあって、もはや時代...
4,movie-enter,2011年の3月11日に起こった東日本大震災。地震、津波の他にも大きな被害をもたらしたものが...


In [56]:
%%time
# BoW
nlp = spacy.load('ja_ginza')
vectorizer = CountVectorizer(analyzer=lambda x: [token.lemma_ for token in nlp(x)])

corpus = summary['text'].values
vectorizer.fit(corpus)

X = vectorizer.transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())

ValueError: [E088] Text of length 1286736 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [45]:
model = TfidfTransformer(smooth_idf=False)

tfidf = model.fit_transform(X)
tfidf.toarray()

array([[0.01984681, 0.02493061, 0.01490467, ..., 0.        , 0.04782828,
        0.04786742],
       [0.01077658, 0.00902468, 0.        , ..., 0.01200431, 0.        ,
        0.        ],
       [0.        , 0.05219698, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.05357912, 0.0224345 , 0.        , ..., 0.02984158, 0.        ,
        0.        ],
       [0.01476719, 0.        , 0.        , ..., 0.0328991 , 0.        ,
        0.        ],
       [0.04510978, 0.1322177 , 0.        , ..., 0.02512447, 0.        ,
        0.        ]])

## modeling

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [57]:
X = np.array(corpus['emb_mean'].to_list())
y = corpus['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=corpus['y'], random_state=5108)

In [58]:
def train_test(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    return y_pred, y_pred_train

models = [LinearSVC(), SVC(), RandomForestClassifier(), DecisionTreeClassifier()]
for model in models:
    y_pred, y_pred_train = train_test(model)
    print(accuracy_score(y_test, y_pred))
    print(accuracy_score(y_train, y_pred_train), '\n')
#     print(classification_report(y_test, y_pred))
#     display(pd.crosstab(y_test, y_pred).style.background_gradient('Blues'))

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.